In [3]:
file_path = "C:\\Users\\johns\\Downloads\\trainingdata.pkl"

In [ ]:
# import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pickle
import time
import sys
import keras
from keras.preprocessing import sequence
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Input, Reshape, BatchNormalization
from keras.layers import (
    Conv1D,
    GlobalAveragePooling1D,
    MaxPooling1D,
    GlobalAveragePooling1D,
    Reshape,
    AveragePooling1D,
    Flatten,
    Concatenate,
)
from keras import backend
from keras.callbacks import TensorBoard, LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

multi_adsorbate = 0  
data_dir = "C:\\Users\\johns\\Downloads\\trainingdata.pkl"  
run_mode = 0  # 0 for regular, 1 for 5-fold CV
split_ratio = 0.2
epochs = 60
batch_size = 32
channels = 2
seed = np.random.randint(1, 1e6)
save_model = True  
load_model_path = 0  
load_model = 0

def main():
    start_time = time.time()

    x_surface_dos, x_adsorbate_dos, y_targets = load_data(
        multi_adsorbate, data_dir
    )

    if run_mode == 0:
        run_training(x_surface_dos, x_adsorbate_dos, y_targets)
    elif run_mode == 1:
        run_kfold(x_surface_dos, x_adsorbate_dos, y_targets)

    print("--- %s seconds ---" % (time.time() - start_time))


def load_data(multi_adsorbate, data_dir):
    with open(file_path, 'rb') as file:
        loaded_data = pickle.load(file)

    target_training = loaded_data['targetd_train']
    features  = loaded_data['featured_train '] 
    DOS_training = [ [list(pair) for pair in zip(*item)] for item in features]
    DOS_feat_training = np.array(DOS_training)
    targets = np.array(target_training)
    surface_dos = DOS_feat_training.astype(np.float32)
    print(targets,surface_dos)
    return surface_dos, None, targets


### changed to take one input instead of three 
def create_model(channels):
    input_layer = Input(shape=(2000, channels))
    x = Conv1D(64, 3, activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling1D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='linear')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# altered 
def create_model_combined(shared_conv, channels):
    ###Each input represents one out of three possible bonding atoms
    input1 = Input(shape=(2000, channels))
    input2 = Input(shape=(2000, channels))
    input3 = Input(shape=(2000, channels))
    input4 = Input(shape=(2000, channels))

    conv1 = shared_conv(input1)
    conv2 = shared_conv(input2)
    conv3 = shared_conv(input3)

    adsorbate_conv = adsorbate_dos_featurizer(channels)
    conv4 = adsorbate_conv(input4)

    convmerge = Concatenate(axis=-1)([conv1, conv2, conv3, conv4])
    convmerge = Flatten()(convmerge)
    convmerge = Dropout(0.2)(convmerge)
    convmerge = Dense(400, activation="linear")(convmerge) # change 
    convmerge = Dense(1000, activation="relu")(convmerge)
    convmerge = Dense(1000, activation="relu")(convmerge)

    out = Dense(1, activation="linear")(convmerge)
    model = Model(inputs=[input1, input2, input3, input4], outputs=out)
    return model



###This sub-model is the convolutional network for the DOS
###Input is a 2000 length DOS data series
def dos_featurizer(channels):
    input_dos = Input(shape=(2000, channels))
    x1 = AveragePooling1D(pool_size=4, strides=4, padding="same")(input_dos)
    x2 = AveragePooling1D(pool_size=25, strides=4, padding="same")(input_dos)
    x3 = AveragePooling1D(pool_size=200, strides=4, padding="same")(input_dos)
    x = Concatenate(axis=-1)([x1, x2, x3])
    x = Conv1D(50, 20, activation="relu", padding="same", strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(75, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(100, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(125, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(150, 3, activation="relu", padding="same", strides=1)(x)
    shared_model = Model(input_dos, x)
    return shared_model


###Uses the same model for adsorbate but w/ separate weights
def adsorbate_dos_featurizer(channels):
    input_dos = Input(shape=(2000, channels))
    x1 = AveragePooling1D(pool_size=4, strides=4, padding="same")(input_dos)
    x2 = AveragePooling1D(pool_size=25, strides=4, padding="same")(input_dos)
    x3 = AveragePooling1D(pool_size=200, strides=4, padding="same")(input_dos)
    x = Concatenate(axis=-1)([x1, x2, x3])
    x = Conv1D(50, 20, activation="relu", padding="same", strides=2)(x)
    x = BatchNormalization()(x)
    x = Conv1D(75, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(100, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(125, 3, activation="relu", padding="same", strides=2)(x)
    x = AveragePooling1D(pool_size=3, strides=2, padding="same")(x)
    x = Conv1D(150, 3, activation="relu", padding="same", strides=1)(x)
    shared_model = Model(input_dos, x)
    return shared_model


###Simple learning rate scheduler
def decay_schedule(epoch, lr):
    if epoch == 0:
        lr = 0.001
    elif epoch == 15:
        lr = 0.0005
    elif epoch == 35:
        lr = 0.0001
    elif epoch == 45:
        lr = 0.00005
    elif epoch == 55:
        lr = 0.00001
    return lr

def run_training(x_surface_dos, x_adsorbate_dos, y_targets):
    x_train, x_test, y_train, y_test = train_test_split(
        x_surface_dos, y_targets, test_size=split_ratio, random_state=88
    )

    # Scaling data
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
    x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

    # Call and fit model
    model = create_model(channels)
    model.compile(loss="logcosh", optimizer=Adam(0.001), metrics=["mean_absolute_error"])

    lr_scheduler = LearningRateScheduler(decay_schedule, verbose=0)
    tensorboard = TensorBoard(log_dir="logs/{}".format(time.time()), histogram_freq=1)

    model.fit(
        x_train, y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        callbacks=[tensorboard, lr_scheduler]
    )

    # Evaluate the model
    train_out = model.predict(x_train)
    test_out = model.predict(x_test)

    print("train MAE: ", mean_absolute_error(y_train, train_out))
    print("train RMSE: ", np.sqrt(mean_squared_error(y_train, train_out)))
    print("test MAE: ", mean_absolute_error(y_test, test_out))
    print("test RMSE: ", np.sqrt(mean_squared_error(y_test, test_out)))

    if save_model:
        model.save("DOSnet_simplified.h5")


# kfold
def run_kfold(x_surface_dos, x_adsorbate_dos, y_targets):
    cvscores = []
    count = 0
    kfold = KFold(n_splits=5, shuffle=True, random_state=seed)

    for train, test in kfold.split(x_surface_dos, y_targets):

        scaler_CV = StandardScaler()
        x_surface_dos[train, :, :] = scaler_CV.fit_transform(
            x_surface_dos[train, :, :].reshape(-1, x_surface_dos[train, :, :].shape[-1])
        ).reshape(x_surface_dos[train, :, :].shape)
        x_surface_dos[test, :, :] = scaler_CV.transform(
            x_surface_dos[test, :, :].reshape(-1, x_surface_dos[test, :, :].shape[-1])
        ).reshape(x_surface_dos[test, :, :].shape)
        if multi_adsorbate == 1:
            x_adsorbate_dos[train, :, :] = scaler.fit_transform(
                x_adsorbate_dos[train, :, :].reshape(
                    -1, x_adsorbate_dos[train, :, :].shape[-1]
                )
            ).reshape(x_adsorbate_dos[train, :, :].shape)
            x_adsorbate_dos[test, :, :] = scaler.transform(
                x_adsorbate_dos[test, :, :].reshape(
                    -1, x_adsorbate_dos[test, :, :].shape[-1]
                )
            ).reshape(x_adsorbate_dos[test, :, :].shape)

        keras.backend.clear_session()
        shared_conv = dos_featurizer(channels)
        lr_scheduler = LearningRateScheduler(decay_schedule, verbose=0)
        if multi_adsorbate == 0:
            model_CV = create_model(shared_conv, channels)
            model_CV.compile(
                loss="logcosh", optimizer=Adam(0.001), metrics=["mean_absolute_error"]
            )
            model_CV.fit(
                [
                    x_surface_dos,
                ],
                y_targets[train],
                batch_size=batch_size,
                epochs=epochs,
                verbose=0,
                callbacks=[lr_scheduler],
            )
            scores = model_CV.evaluate(
                [
                    x_surface_dos,
                ],
                y_targets[test],
                verbose=0,
            )
            train_out_CV_temp = model_CV.predict(
                [
                    x_surface_dos,
                ]
            )
            train_out_CV_temp = train_out_CV_temp.reshape(len(train_out_CV_temp))
        elif multi_adsorbate == 1:
            model_CV = create_model_combined(shared_conv, channels)
            model_CV.compile(
                loss="logcosh", optimizer=Adam(0.001), metrics=["mean_absolute_error"]
            )
            model_CV.fit(
                [
                    x_surface_dos,
                    x_adsorbate_dos[train, :, :],
                ],
                y_targets[train],
                batch_size=batch_size,
                epochs=epochs,
                verbose=0,
                callbacks=[lr_scheduler],
            )
            scores = model_CV.evaluate(
                [
                    x_surface_dos,
                    x_adsorbate_dos[test, :, :],
                ],
                y_targets[test],
                verbose=0,
            )
            train_out_CV_temp = model_CV.predict(
                [
                    x_surface_dos,
                    x_adsorbate_dos[test, :, :],
                ]
            )
            train_out_CV_temp = train_out_CV_temp.reshape(len(train_out_CV_temp))
        print((model_CV.metrics_names[1], scores[1]))
        cvscores.append(scores[1])
        if count == 0:
            train_out_CV = train_out_CV_temp
            test_y_CV = y_targets[test]
            test_index = test
        elif count > 0:
            train_out_CV = np.append(train_out_CV, train_out_CV_temp)
            test_y_CV = np.append(test_y_CV, y_targets[test])
            test_index = np.append(test_index, test)
        count = count + 1
    print((np.mean(cvscores), np.std(cvscores)))
    print(len(test_y_CV))
    print(len(train_out_CV))
    with open("CV_predict.txt", "w") as f:
        np.savetxt(f, np.stack((test_y_CV, train_out_CV), axis=-1))


if __name__ == "__main__":
    main()